In [172]:
from pyspark.sql import SparkSession

def _spark_context():
    'Creates a local spark context'

    return SparkSession.builder \
      .master('local') \
      .appName('syllabus') \
      .getOrCreate()

SPARK = _spark_context()
SPARK

In [173]:
import json

from pyspark.sql import DataFrame
from pyspark.sql import functions as F
from pyspark.sql import types as T

from pygments import highlight
from pygments.lexers import JsonLexer
from pygments.formatters import TerminalTrueColorFormatter

def ppj(j, indent=2):
    print(highlight(
        code      = json.dumps(json.loads(j), indent=indent),
        lexer     = JsonLexer(),
        formatter = TerminalTrueColorFormatter()
    ).strip())
   
def ppd(d, indent=2):
    ppj(json.dumps(d), indent=indent)

def count_nulls(df: DataFrame) -> int:
    return df.select(
        sum([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df.columns])
    ).collect()[0][0]

def count_cells(df: DataFrame) -> int:
    return df.count() * len(df.columns)

class DFLoader:
    @staticmethod
    def from_file(records: list, fpath: str = 'f.ndjson', schema: dict = {}) -> DataFrame:
        with open(fpath, 'w') as ostream:
            for record in records:
                print(json.dumps(record), file=ostream, end='\n')
        if schema:
            df = SPARK.read.json(fpath, schema=T.StructType.fromJson(schema))
        else:
            df = SPARK.read.json(fpath)
        df.show()
        print('cells', count_cells(df), '/', 'nulls', count_nulls(df))
        ppj(df.schema.json())
        return df

In [174]:
records = [
    { "a": "a", "c": "d" },
    { "a": "b" },
    { "a": "c" },
    { "a": 1, "d": "z" }
]
df = DFLoader.from_file(records)
count_nulls(df)

+---+----+----+
|  a|   c|   d|
+---+----+----+
|  a|   d|null|
|  b|null|null|
|  c|null|null|
|  1|null|   z|
+---+----+----+

cells 12 / nulls 6
{
  "fields": [
    {
      "metadata": {},
      "name": "a",
      "nullable": true,
      "type": "string"
    },
    {
      "metadata": {},
      "name": "c",
      "nullable": true,
      "type": "string"
    },
    {
      "metadata": {},
      "name": "d",
      "nullable": true,
      "type": "string"
    }
  ],
  "type": "struct"
}


6

In [175]:
records = [
    { "a": 1, "c": "d" },
    { "a": "b" },
    { "a": "c" },
    { "a": "123", "d": "z" }
]
df = DFLoader.from_file(records)

+---+----+----+
|  a|   c|   d|
+---+----+----+
|  1|   d|null|
|  b|null|null|
|  c|null|null|
|123|null|   z|
+---+----+----+

cells 12 / nulls 6
{
  "fields": [
    {
      "metadata": {},
      "name": "a",
      "nullable": true,
      "type": "string"
    },
    {
      "metadata": {},
      "name": "c",
      "nullable": true,
      "type": "string"
    },
    {
      "metadata": {},
      "name": "d",
      "nullable": true,
      "type": "string"
    }
  ],
  "type": "struct"
}


In [176]:
records = [
    { "id": 123, "a": 1, "c": "d" },
    { "id": 122, "a": "b" },
    { "id": 111, "a": "c" },
    { "id": 234, "a": 1, "d": "z" }
]
df = DFLoader.from_file(records)

+---+----+----+---+
|  a|   c|   d| id|
+---+----+----+---+
|  1|   d|null|123|
|  b|null|null|122|
|  c|null|null|111|
|  1|null|   z|234|
+---+----+----+---+

cells 16 / nulls 6
{
  "fields": [
    {
      "metadata": {},
      "name": "a",
      "nullable": true,
      "type": "string"
    },
    {
      "metadata": {},
      "name": "c",
      "nullable": true,
      "type": "string"
    },
    {
      "metadata": {},
      "name": "d",
      "nullable": true,
      "type": "string"
    },
    {
      "metadata": {},
      "name": "id",
      "nullable": true,
      "type": "long"
    }
  ],
  "type": "struct"
}


In [177]:
records = [
    { "id": 123, "key": "a", "value": 1},
    { "id": 123, "key": "c", "value": "d" },
    { "id": 122, "key": "a", "value": "b" },
    { "id": 111, "key": "a", "value": "c" },
    { "id": 234, "key": "a", "value": 1 },
    { "id": 234, "key": "d", "value": "z" },
]
df = DFLoader.from_file(records)

+---+---+-----+
| id|key|value|
+---+---+-----+
|123|  a|    1|
|123|  c|    d|
|122|  a|    b|
|111|  a|    c|
|234|  a|    1|
|234|  d|    z|
+---+---+-----+

cells 18 / nulls 0
{
  "fields": [
    {
      "metadata": {},
      "name": "id",
      "nullable": true,
      "type": "long"
    },
    {
      "metadata": {},
      "name": "key",
      "nullable": true,
      "type": "string"
    },
    {
      "metadata": {},
      "name": "value",
      "nullable": true,
      "type": "string"
    }
  ],
  "type": "struct"
}


In [178]:
records = [
    { "id": 123, "a": "1", "c": "d" },
    { "id": 122, "a": "b" },
    { "id": 111, "a": "c" },
    { "id": 234, "a": "z", "d": "z" }
]

schema = {
    "type" : "object",
    "properties" : {
        "id" : {"type" : "integer"},
        "a": {"type": "string"},
    },
    "required": ["id"],
    "additionalProperties": False
}

print("--- records")
ppd(records)
print("--- schema")
ppd(schema)

--- records
[
  {
    "id": 123,
    "a": "1",
    "c": "d"
  },
  {
    "id": 122,
    "a": "b"
  },
  {
    "id": 111,
    "a": "c"
  },
  {
    "id": 234,
    "a": "z",
    "d": "z"
  }
]
--- schema
{
  "type": "object",
  "properties": {
    "id": {
      "type": "integer"
    },
    "a": {
      "type": "string"
    }
  },
  "required": [
    "id"
  ],
  "additionalProperties": false
}


In [179]:
from typing import Dict, List
from jsonschema.exceptions import ValidationError
from jsonschema.validators import Draft3Validator
from jsonschema import validate

def translate(df: dict) -> dict:
    v = Draft3Validator(schema)
    for record in df:
        for error in sorted(v.iter_errors(record), key=str):
            yield record, error

def parse_unexpected_keys(instance: dict, e: ValidationError) -> List[str]:
    '''
    "Additional properties are not allowed ('a' was unexpected)" -> ['a']
    "Additional properties are not allowed ('a', 'c' were unexpected)" -> ['a', 'c']
    '''
    return (
        list(map(
            # Strip the single quote from the outside of each key
            lambda x: x.strip("'"),
            # Strip the message of the beginning/end, 
            e.args[0]
            .lstrip('Additional properties are not allowed (')
            .rstrip(' was unexpected)')
            .rstrip(' were unexpected)')
            # split on comma e.g. "('a', 'c' was...)" -> ["'a'", "'b'"]
            .split(', ')
        ))
    )
            
for record, error in translate(records):
    ppd({
        'instance': record,
        'error': error.message,
        'keys': parse_unexpected_keys(record, error)
    })

{
  "instance": {
    "id": 123,
    "a": "1",
    "c": "d"
  },
  "error": "Additional properties are not allowed ('c' was unexpected)",
  "keys": [
    "c"
  ]
}
{
  "instance": {
    "id": 234,
    "a": "z",
    "d": "z"
  },
  "error": "Additional properties are not allowed ('d' was unexpected)",
  "keys": [
    "d"
  ]
}


In [180]:
from dataclasses import dataclass, field

@dataclass
class UnexpectedKeys:
    schema: dict
    unique_keys: set = field(default_factory=set)
    
    def __post_init__(self):
        self.validator = Draft3Validator(self.schema)

    @property
    def keys(self):
        return list(sorted(self.unique_keys))

    @staticmethod
    def parse_unexpected_keys(e: ValidationError) -> List[str]:
        '''
        "Additional properties are not allowed ('a' was unexpected)" -> ['a']
        "Additional properties are not allowed ('a', 'c' were unexpected)" -> ['a', 'c']
        '''
        return (
            list(map(
                # Strip the single quote from the outside of each key
                lambda x: x.strip("'"),
                # Strip the message of the beginning/end, 
                e.args[0]
                .lstrip('Additional properties are not allowed (')
                .rstrip(' was unexpected)')
                .rstrip(' were unexpected)')
                # split on comma e.g. "('a', 'c' was...)" -> ["'a'", "'b'"]
                .split(', ')
            ))
        )

    
    def check(self, instance: dict):
        for error in sorted(self.validator.iter_errors(instance), key=str):
            for key in UnexpectedKeys.parse_unexpected_keys(error):
                self.unique_keys.add(key)
            yield error

In [181]:
k = UnexpectedKeys(schema)
for record in records:
    for error in k.check(record):
        print('--- record')
        ppd(record)
        print('--- error', error, sep='\n')

--- record
{
  "id": 123,
  "a": "1",
  "c": "d"
}
--- error
Additional properties are not allowed ('c' was unexpected)

Failed validating 'additionalProperties' in schema:
    {'additionalProperties': False,
     'properties': {'a': {'type': 'string'}, 'id': {'type': 'integer'}},
     'required': ['id'],
     'type': 'object'}

On instance:
    {'a': '1', 'c': 'd', 'id': 123}
--- record
{
  "id": 234,
  "a": "z",
  "d": "z"
}
--- error
Additional properties are not allowed ('d' was unexpected)

Failed validating 'additionalProperties' in schema:
    {'additionalProperties': False,
     'properties': {'a': {'type': 'string'}, 'id': {'type': 'integer'}},
     'required': ['id'],
     'type': 'object'}

On instance:
    {'a': 'z', 'd': 'z', 'id': 234}


In [182]:
k.unique_keys

{'c', 'd'}

In [183]:
k.unique_keys

def split_dict(d: dict, keys: list):
    dd, extras = {}, {}
    for k in d.keys():
        if k in keys:
            extras[k] = d[k]
        else:
            dd[k] = d[k]
    return dd, extras

def translate(df: list) -> list:
    uk = UnexpectedKeys(schema)
    for record in records:
        for error in uk.check(record):
            d, extras = split_dict(record, UnexpectedKeys.parse_unexpected_keys(error))
            if extras:
                for k, v in extras.items():
                    yield {
                        **d, **{"custom": [{"key": k, "value": v}]}
                    }
            else:
                yield {**d, **{"custom": []}}
            break
        else:
            yield {**record, **{"custom": []}}

In [184]:
print('--- records')
for record in records:
    ppd(record, indent=None)
print('--- translated')
for record in translate(records):
    ppd(record, indent=None)

--- records
{"id": 123, "a": "1", "c": "d"}
{"id": 122, "a": "b"}
{"id": 111, "a": "c"}
{"id": 234, "a": "z", "d": "z"}
--- translated
{"id": 123, "a": "1", "custom": [{"key": "c", "value": "d"}]}
{"id": 122, "a": "b", "custom": []}
{"id": 111, "a": "c", "custom": []}
{"id": 234, "a": "z", "custom": [{"key": "d", "value": "z"}]}


In [185]:
schema = {
    "type" : "object",
    "properties" : {
        "id" : {"type" : "integer"},
        "a": {"type": "string"},
        "custom": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "key": {"type": "string"},
                    "value": {"type": "string"},
                }
            }
        }
    },
    "required": ["id", "custom"],
    "additionalProperties": False
}
ppd(schema)

{
  "type": "object",
  "properties": {
    "id": {
      "type": "integer"
    },
    "a": {
      "type": "string"
    },
    "custom": {
      "type": "array",
      "items": {
        "type": "object",
        "properties": {
          "key": {
            "type": "string"
          },
          "value": {
            "type": "string"
          }
        }
      }
    }
  },
  "required": [
    "id",
    "custom"
  ],
  "additionalProperties": false
}


In [186]:
for record in translate(records):
    ppd(record, indent=None)
    validate(record, schema)

{"id": 123, "a": "1", "custom": [{"key": "c", "value": "d"}]}
{"id": 122, "a": "b", "custom": []}
{"id": 111, "a": "c", "custom": []}
{"id": 234, "a": "z", "custom": [{"key": "d", "value": "z"}]}


In [187]:
records = [
    { "id": 123, "a": "1", "c": "d", "z": "zz" },
    { "id": 122, "a": "b" },
    { "id": 111, "a": "c" },
    { "id": 234, "a": "z", "d": "z" }
]

print('--- records')
for record in records:
    ppd(record, indent=None)

print('--- translated')
for record in translate(records):
    ppd(record, indent=None)

--- records
{"id": 123, "a": "1", "c": "d", "z": "zz"}
{"id": 122, "a": "b"}
{"id": 111, "a": "c"}
{"id": 234, "a": "z", "d": "z"}
--- translated
{"id": 123, "a": "1", "custom": [{"key": "c", "value": "d"}]}
{"id": 123, "a": "1", "custom": [{"key": "z", "value": "zz"}]}
{"id": 122, "a": "b", "custom": []}
{"id": 111, "a": "c", "custom": []}
{"id": 234, "a": "z", "custom": [{"key": "d", "value": "z"}]}


In [188]:
df = DFLoader.from_file(translate(records))

+---+---------+---+
|  a|   custom| id|
+---+---------+---+
|  1| [{c, d}]|123|
|  1|[{z, zz}]|123|
|  b|       []|122|
|  c|       []|111|
|  z| [{d, z}]|234|
+---+---------+---+

cells 15 / nulls 0
{
  "fields": [
    {
      "metadata": {},
      "name": "a",
      "nullable": true,
      "type": "string"
    },
    {
      "metadata": {},
      "name": "custom",
      "nullable": true,
      "type": {
        "containsNull": true,
        "elementType": {
          "fields": [
            {
              "metadata": {},
              "name": "key",
              "nullable": true,
              "type": "string"
            },
            {
              "metadata": {},
              "name": "value",
              "nullable": true,
              "type": "string"
            }
          ],
          "type": "struct"
        },
        "type": "array"
      }
    },
    {
      "metadata": {},
      "name": "id",
      "nullable": true,
      "type": "long"
    }
  ],
  "type": "struct

In [189]:
df.select(df.id, df.a, df.custom.key, df.custom.value).show()

+---+---+----------+------------+
| id|  a|custom.key|custom.value|
+---+---+----------+------------+
|123|  1|       [c]|         [d]|
|123|  1|       [z]|        [zz]|
|122|  b|        []|          []|
|111|  c|        []|          []|
|234|  z|       [d]|         [z]|
+---+---+----------+------------+



In [190]:
def translate2(df: list) -> list:
    uk = UnexpectedKeys(schema)
    for record in records:
        for error in uk.check(record):
            d, extras = split_dict(record, UnexpectedKeys.parse_unexpected_keys(error))
            custom = []
            if extras:
                for k, v in extras.items():
                    custom.append({"key": k, "value": v})
                yield {**d, **{"custom": custom}}
            else:
                yield {**d, **{"custom": custom}}
            break
        else:
            yield {**record, **{"custom": []}}

In [191]:
records = [
    { "id": 123, "a": "1", "c": "d", "z": "zz" },
    { "id": 122, "a": "b" },
    { "id": 111, "a": "c" },
    { "id": 234, "a": "z", "d": "z" }
]

print('--- records')
for record in records:
    ppd(record, indent=None)

print('--- translated')
for record in translate2(records):
    ppd(record, indent=None)

--- records
{"id": 123, "a": "1", "c": "d", "z": "zz"}
{"id": 122, "a": "b"}
{"id": 111, "a": "c"}
{"id": 234, "a": "z", "d": "z"}
--- translated
{"id": 123, "a": "1", "custom": [{"key": "c", "value": "d"}, {"key": "z", "value": "zz"}]}
{"id": 122, "a": "b", "custom": []}
{"id": 111, "a": "c", "custom": []}
{"id": 234, "a": "z", "custom": [{"key": "d", "value": "z"}]}


In [192]:
df = DFLoader.from_file(translate2(records))

+---+-----------------+---+
|  a|           custom| id|
+---+-----------------+---+
|  1|[{c, d}, {z, zz}]|123|
|  b|               []|122|
|  c|               []|111|
|  z|         [{d, z}]|234|
+---+-----------------+---+

cells 12 / nulls 0
{
  "fields": [
    {
      "metadata": {},
      "name": "a",
      "nullable": true,
      "type": "string"
    },
    {
      "metadata": {},
      "name": "custom",
      "nullable": true,
      "type": {
        "containsNull": true,
        "elementType": {
          "fields": [
            {
              "metadata": {},
              "name": "key",
              "nullable": true,
              "type": "string"
            },
            {
              "metadata": {},
              "name": "value",
              "nullable": true,
              "type": "string"
            }
          ],
          "type": "struct"
        },
        "type": "array"
      }
    },
    {
      "metadata": {},
      "name": "id",
      "nullable": true,
     

In [193]:
df.select(df.id, df.a, df.custom.key, df.custom.value).show()

+---+---+----------+------------+
| id|  a|custom.key|custom.value|
+---+---+----------+------------+
|123|  1|    [c, z]|     [d, zz]|
|122|  b|        []|          []|
|111|  c|        []|          []|
|234|  z|       [d]|         [z]|
+---+---+----------+------------+



In [194]:
def select_rows_with_custom_key(df: DataFrame, key: str) -> DataFrame:
    return (
        df.where(
            F.array_contains(df.custom.key,  key)
        )
        .select(
            df.id,
            df.a,
            df.custom.key,
            df.custom.value
        )
    )

In [195]:
k = UnexpectedKeys(schema)
for record in records:
    list(k.check(record))

for key in k.unique_keys:
    print("--- custom key", key)
    display(select_rows_with_custom_key(df, key).toPandas())

--- custom key c


,id,a,custom.key,custom.value
0,123,1,"[c, z]","[d, zz]"


--- custom key z


,id,a,custom.key,custom.value
0,123,1,"[c, z]","[d, zz]"


--- custom key d


,id,a,custom.key,custom.value
0,234,z,[d],[z]


In [196]:
# select_rows_with_custom_key(df, 'c').select(F.filter(df.custom

In [198]:
schema = {
  "fields": [
    {
      "metadata": {},
      "name": "a",
      "nullable": True,
      "type": "string"
    },
    {
      "metadata": {},
      "name": "custom",
      "nullable": True,
      "type": {
        "containsNull": True,
        "elementType": {
          "fields": [
            {
              "metadata": {},
              "name": "key",
              "nullable": True,
              "type": "string"
            },
            {
              "metadata": {},
              "name": "value",
              "nullable": True,
              "type": "string"
            }
          ],
          "type": "struct"
        },
        "type": "array"
      }
    },
    {
      "metadata": {},
      "name": "id",
      "nullable": True,
      "type": "long"
    }
  ],
  "type": "struct"
}

In [199]:
df = DFLoader.from_file(translate(records), schema=schema)

UnknownType: Unknown type 'struct' for validator with schema:
    {'fields': [{'metadata': {},
                 'name': 'a',
                 'nullable': True,
                 'type': 'string'},
                {'metadata': {},
                 'name': 'custom',
                 'nullable': True,
                 'type': {'containsNull': True,
                          'elementType': {'fields': [{'metadata': {},
                                                      'name': 'key',
                                                      'nullable': True,
                                                      'type': 'string'},
                                                     {'metadata': {},
                                                      'name': 'value',
                                                      'nullable': True,
                                                      'type': 'string'}],
                                          'type': 'struct'},
                     

In [ ]:
for key in k.unique_keys:
    print("--- custom key", key)
    display(select_rows_with_custom_key(df, key).toPandas())

In [ ]:
c = select_rows_with_custom_key(df, 'c')
c.select(c.custom.value)